In [13]:
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
churn_data = pd.read_csv('data/churn.csv')
churn_data = churn_data.drop('RowNumber', axis=1)
churn_data.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [14]:
loyalty = churn_data['Exited'].value_counts()

fig = px.pie(names=['loyal', 'lost'],
             values=loyalty,
             title='Customer Loyalty',
             width=500,
             height=400,
)
fig.show()

9.1. Исходя из представленных данных, примерно каждый пятый клиент покинул банк.

In [15]:
clients_2500 = churn_data[churn_data['Balance'] > 2500]

fig = px.histogram(clients_2500,
                   x='Balance',
                   color_discrete_sequence=['limegreen'],
                   title='Balance Distribution',
                   nbins=100,
                   width=647,
                   height=400                  
)
fig.layout.yaxis.title.text = 'Number of clients'
fig.show()

9.2. Основная масса клиентов имеют на счетах суммы примерно от 50 до 200 тысяч, наиболее типичная сумма (пик распределения) составляет 105-130 тысяч. У банка нет клиентов-миллионеров - максимальная сумма на счете порядка 250 тысяч. Распределение балансов в целом похоже на нормальное.

In [16]:
clients_2500 = clients_2500.copy()
clients_2500['Loyalty'] = clients_2500['Exited'].apply(lambda x: 'loyal' if x==0 else 'lost')

fig = px.box(clients_2500, 
             x='Balance',
             color='Loyalty',
             title='Loyalty Depending on the Balance',
             width=647,
             height=400
)
fig.layout.legend.title = None
fig.show()

9.3. Медианные значения распределений практически не отличаются, однако распределение ушедших клиентов более узкое по сравнению с распределением лояльных: его границы лежат ближе к медиане. Можно сказать, что среди клиентов "среднего класса" тенденция ухода выше. Возможно, банк уделяет недостаточно внимания удержанию этой наиболее многочисленной категории, а конкуренты предлагают более выгодные условия для счетов среднего размера. В то же время стоит отметить, что из банка ушли как два самых "богатых", так и два самых "бедных" клиента - крайние точки аномальных выбросов на распределении ушедших клиентов расположены шире, чем на распределении лояльных, однако эти единичные случаи не влияют на общую картину.

In [17]:
fig = px.histogram(clients_2500,
                   x='Age',
                   title='Loyalty Depending on the Age',
                   color='Loyalty',
                   barmode='group',
                   marginal='box',
                   nbins=100,
                   width=647,
                   height=400                  
)
fig.layout.yaxis.title.text = 'Number of clients'
fig.layout.legend.title = None
fig.show()

9.4. Наблюдаются значительные возрастные отличия между группами лояльных и нелояльных клиентов. Надежными клиентами банка являются люди в возрасте 30-40 лет - эта возрастная категория формирует наиболее многочисленную группу лояльных клиентов. Распределение лояльных клиентов также показывает большое количество аномальных выбросов (хвост распределения) для старшего возраста 55 и более лет, что может означать, что пожилые люди также удовлетворены банком, однако их количество относительно мало. Особое внимание следует уделить возрастной категории 40-50 лет - люди именно этого возраста чаще всего уходят из банка, однако долговременную заинтересованность стоит начинать формировать раньше, до возраста 38-39 лет, с которого начинается тенденция потери интереса к услугам банка.

In [18]:
fig = px.scatter(clients_2500,
                 x='EstimatedSalary',
                 y='CreditScore',
                 color='Loyalty',
                 labels={'EstimatedSalary': 'Estimated salary', 'CreditScore': 'Credit score'},
                 title='Credit Score vs Salary',
                 height=400                
)
fig.layout.legend.title = None
fig.show()

9.5. Все клиенты с предельно низким кредитным рейтингом (менее 400) покинули банк. Можно заметить концентрацию к значениям рейтинга 500-800 независимо от уровня заработной платы и лояльности. В остальном закономерностей и взаимосвязи между признаками не наблюдается. Виден "потолок" максимально возможного кредитного рейтинга - 850, есть довольно большое количество клиентов с максимальным рейтингом.

In [19]:
by_gender = clients_2500.groupby('Gender', as_index=False)['Exited'].mean()
by_gender['Exited'] = by_gender['Exited'] * 100

fig = px.bar(by_gender,
             x='Gender',
             y='Exited',
             color='Gender',
             color_discrete_map={'Female': 'LightCoral', 'Male': 'LightSeaGreen'},
             labels={'Exited': 'Lost clients, %'},
             title='Lost Clients by Gender',
             width=247,
             height=400 
)
fig.update_layout(showlegend=False)
fig.update_xaxes(tickangle=30)
fig.show()

9.6. Женщины гораздо чаще покидают банк: 30% ушедших клиентов против 20% среди мужчин. Цифры немного не согласуются с указанными в первом рисунке (Customer Loyalty) из-за того, что сейчас мы рассматриваем только клиентов с балансом счета более 2500.

In [20]:
by_products = clients_2500.groupby('NumOfProducts')['Exited'].agg(['sum', 'count']).reset_index()
by_products.rename(columns={'sum': 'lost'}, inplace=True)
by_products['loyal'] = by_products['count'] - by_products['lost']
by_products.drop('count', axis=1, inplace=True)

fig = px.bar(by_products, 
             x="NumOfProducts", 
             y=["loyal", "lost"],
             barmode='group',
             title='Loyalty Depending on the Number of Products',
             width=490,
             height=400
)
fig.layout.xaxis.title = 'Number of products'
fig.layout.yaxis.title = 'Number of clients'
fig.layout.legend.title = None
fig.show()

9.7. Наименьший относительный отток наблюдается среди клиентов, которые приобрели две услуги. При этом количество клиентов с одной услугой больше, но и относительный отток у них выше (уходит четверть клиентов). Почти все клиенты, которые приобрели 3 или 4 услуги, покинули банк - у этих категорий относительный отток максимальный, близкий к 100%, но общее количество этих клиентов не так велико.

In [21]:
by_activity = clients_2500.groupby('IsActiveMember')['Exited'].agg(['sum', 'count']).reset_index()
by_activity.rename(columns={'sum': 'lost'}, inplace=True)
by_activity['loyal'] = by_activity['count'] - by_activity['lost']
by_activity.drop('count', axis=1, inplace=True)
by_activity.loc[0, 'IsActiveMember'] = 'inactive'
by_activity.loc[1, 'IsActiveMember'] = 'active'

fig = px.bar(by_activity, 
             x="IsActiveMember", 
             y=["loyal", "lost"],
             barmode='group',
             title='Loyalty Depending on the Activity',
             width=323,
             height=400
)
fig.layout.xaxis.title = None
fig.layout.yaxis.title = 'Number of Clients'
fig.layout.legend.title = None
fig.show()

 9.8. У банка довольно большое количество неактивных клиентов, среди которых отток значительно больше, чем среди активных. Уменьшать отток среди неактивных клиентов (пардон, но так сформулирован вопрос к заданию), на мой взгляд, довольно бесперспективная идея, т.к. отсутствие активности в общем случае сигнализирует об отсутствии интереса. Более разумно стимулировать клиентов к переходу в разряд активных. Сделать это можно, например, предоставлением различных бонусов за операции по карте: кэшбэк, процент на остаток по расчетному счету, бонусы на приобретение услуг у партнеров, - а также развитием сервиса по проведению различных платежей - страховки, билеты, переводы, предоставлением удобного доступа к различным финансовым инструментам и пр. В целом, для выставления каких-либо рекомендаций необходимо хорошо знать уже существующие программы лояльности банка и инструменты взаимодействия с клиентами, а также анализировать предложения и преимущества конкурентов.

In [22]:
by_country = clients_2500.groupby('Geography', as_index=False)['Exited'].mean()
by_country['Churn rate, %'] = by_country['Exited'] * 100

fig = px.choropleth(
    data_frame=by_country, 
    locations="Geography", 
    locationmode = "country names", 
    color="Churn rate, %", 
    range_color=[0, 50],
    title='Geography of Lost Clients', 
    scope='europe',
    fitbounds='locations',
    width=500, 
    height=500,
    color_continuous_scale='Reds'
)
fig.show()

9.9. Наибольший отток клиентов в Германии: более 30%. Поскольку все три страны примерно одинаковы по уровню благосостояния, вероятными причинами могут быть появление в Германии более привлекательного национального банка, предлагающего более подходящие решения для граждан Германии. Также возможно снижение лояльности в силу политических причин.

In [23]:
def get_credit_score_cat(credit_score):
    if credit_score >= 300 and credit_score < 500:
        return "Very poor"
    elif credit_score >= 500 and credit_score < 601:
        return "Poor"
    elif credit_score >= 601 and credit_score < 661:
        return "Fair"
    elif credit_score >= 661 and credit_score < 781:
        return "Good"
    elif credit_score >= 781 and credit_score < 851:
        return "Excellent"
    elif credit_score >= 851:
        return "Top"
    elif credit_score < 300:
        return "Deep"
    

clients_2500['CreditScoreCat'] = clients_2500['CreditScore'].apply(get_credit_score_cat)
by_score_tenure = pd.pivot_table(
    clients_2500,
    values='Exited',
    index='CreditScoreCat',
    columns='Tenure',
    aggfunc='mean'                             
)
by_score_tenure = (by_score_tenure * 100).round(1)

fig = ff.create_annotated_heatmap(
    z=by_score_tenure.to_numpy(),
    x=by_score_tenure.columns.tolist(),
    y=by_score_tenure.index.tolist(), 
    colorscale='Reds',
    zmin=0, zmax=50,
    showscale=True   
)
fig.add_annotation(x=1.17,
                   y=1.09,
                   showarrow=False,
                   text="Churn rate, %",
                   xref="paper",
                   yref="paper"
)
fig.update_layout(width=647, height=400)
fig.layout.xaxis.title = 'Tenure'
fig.layout.yaxis.title = 'Credit score'
fig.layout.title = 'Churn Rate Heatmap'
fig.show()

9.10. Самый большой отток (более 45%) у клиентов с очень плохим кредитным рейтингом и минимальным стажем пользования услугами банка (менее года). За ними следуют клиенты с таким же очень плохим рейтингом, но большим стажем 4 года или 10 лет (отток около 36%). Также довольно большой отток у клиентов с прекрасным рейтингом и стажем 0 или 9 лет (отток около 35%). Можно заметить, что отток более "стабилен" по годам для средних показателей рейтинга, в то время как у клиентов с очень плохим или, наоборот, прекрасным рейтингом отток резко колеблется от минимумов к максимумам в зависимости от количества лет.